In [1]:
import shuo
import fiona
import numpy as np
import networkx as nx
import geopandas as gpd
import matplotlib as mpl
from matplotlib import pyplot as plt
from shapely.geometry import LineString, mapping
from shuo.basics import FileNames, PickleDataWriter as pdw, UnionFind, FilesReader, get_LineString

File_Prefixes = ['Washington_DC', 
                 'Seattle_city_WA',
                 'Chicago_city_IL',
                 'Madison_county_AL', 
                 'Mobile_city_AL', 
                 'Napa_city_CA', 
                 'Redding_city_CA',
                 'Yuma_county_AZ']
budget = 0
current_index = 0
folder = 'data_processed'
file_prefix = File_Prefixes[current_index]
print(file_prefix)

opt = 'minimizing_connected_components'
opt = 'minimizing_infeasible_trips'
F = FilesReader(folder, file_prefix, budget)

Washington_DC


In [2]:
# Load graph to be colored
G = F.graph(3, opt)
cc_num = nx.number_connected_components(G)
print('Colors:', cc_num)

# Get color info for each CC
norm = mpl.colors.Normalize(vmin=0, vmax=cc_num)
color_mapping = mpl.cm.ScalarMappable(norm=norm, cmap=plt.get_cmap('hsv'))
colors = [color_mapping.to_rgba(index + 1) for index in range(cc_num)]

# Get color info of each edge
edge_colors = dict()
cc_sub = nx.connected_component_subgraphs(G)
for index, G in enumerate(cc_sub):
    color = colors[index]
    for u, v, d in G.edges(data=True):
        edge_colors[(u, v)] = index

# Store color info of each edge
for u, v, d in G.edges(data=True):
    d['color'] = str(edge_colors[(u, v)]) if (u, v) in edge_colors else str(edge_colors[(v, u)])

Colors: 284


In [3]:
# Convert the nodes of road network into an easy-to-read format
points = {node: [float(data['x']), float(data['y'])] for node, data in G.nodes(data=True)}

# Create shapefile of colored edges
schema = {'geometry': 'LineString', 'properties':{'color': 'int'}}
with fiona.open(F.name.graph(4, opt), 'w', 'ESRI Shapefile', schema, fiona.crs.from_epsg(4326)) as File:
    for u, v, d in G.edges(data=True):
        elem = dict()
        line = get_LineString(d['geometry']) if 'geometry' in d else LineString([points[u], points[v]])
        elem['geometry'] = mapping(line)
        elem['properties'] = dict()
        elem['properties']['color'] = d['color']

        File.write(elem)